# random action 버전

In [2]:
# On-policy first-visit MC control (p.101)

# TODO 
# 1. performance graph

# Imports
import gym
import numpy as np
from collections import defaultdict

# Map setting
map_desc = ["SFFF", "FHFF", "FFFH", "HFFG"]
# map_desc = ["SFFF", "FFFH", "FFFF", "FFFG"]
env = gym.make('FrozenLake-v1', desc=map_desc, map_name="4x4", is_slippery=False, render_mode='human')

# Constants
NUM_ACTS = env.action_space.n
ETA = 0.1
NUM_EPISODES = 500
GAMMA = 0.9
VERVOSE = False

# Global vars
Q = defaultdict(lambda: np.random.rand(NUM_ACTS))       # {s: [a,...,a]}
Returns = defaultdict(list)                             # {(s, a): [r,...,r]}

# Reporting
tot_rewards = []
tot_steps = []

def random_argmax(value_list):
  """ a random tie-breaking argmax """
  values = np.asarray(value_list)
  return np.argmax(np.random.random(values.shape) * (values==values.max()))

# get action with e-greedy
def get_action(s):
    a = -1
    if np.random.rand() < ETA:
        a = np.random.randint(NUM_ACTS)
        if VERVOSE: print('Random action for %s -> %s' % (s, s))
    else:
        # Find a*
        a = random_argmax(Q[s])
        if VERVOSE: print('Greedy action for %s -> %s' % (s, s))
    return a

for e in range(NUM_EPISODES):
    print("\rEpisode {}/{} --------------------- ".format(e, NUM_EPISODES), end="\n")

    # Env reset
    s, info = env.reset()

    # Terminiation condition
    terminated = False
    truncated = False

    # Trajectory
    traj = []
    num_steps = 0
    fin_reward = 0
    # Generate an episode
    while not terminated and not truncated: 
        # a = np.random.choice(NUM_ACTS, p=PI[s])
        a = get_action(s)
        next_s, r, terminated, truncated, info = env.step(a)       
        traj.append((s, a, r))
        if VERVOSE: print("State:", s, "Action", a, "Reward:", r, "Terminated:", terminated, "Truncated:", truncated, "Info:", info)
        s = next_s
        num_steps += 1
        fin_reward += r
    tot_rewards.append(fin_reward)
    tot_steps.append(num_steps)

    print(f"--> Reward: {fin_reward}, #Steps: {num_steps}, \
                Avg Rewards: {(np.average(tot_rewards))}, Avg steps: {np.average(tot_steps)}")

    # Check trajectory
    if VERVOSE:
        for i, t in enumerate(traj):
            print(i, t)

    # G
    G = 0

    # For each step
    for i, (s, a, r) in reversed(list(enumerate(traj))):
        # Update G
        G = GAMMA*G + r
        first_idx = next(j for j, t in enumerate(traj) if t[0] == s and t[1] == a)
        if VERVOSE: print(i, (s, a, r), first_idx)
        if i == first_idx:
            # print(i, (s, a, r))
            # Append G to (s, a)
            Returns[(s, a)].append(G)
            # Check returns
            if VERVOSE: print('Returns(%s,%s) -> %s' % (s, a, Returns[(s, a)]))            
            # Update Q
            Q[s][a] = sum(Returns[(s, a)]) / len(Returns[(s, a)])
            if VERVOSE: print('Update Q(%s,%s) -> %f' % (s, a, Q[s][a]))

env.close()


'''
Episode 99/100 ---------------------
State: 0 Action 1 Reward: 0.0 Terminated: False Truncated: False Info: {'prob': 1.0}
State: 4 Action 3 Reward: 0.0 Terminated: False Truncated: False Info: {'prob': 1.0}
State: 0 Action 1 Reward: 0.0 Terminated: False Truncated: False Info: {'prob': 1.0}
State: 4 Action 1 Reward: 0.0 Terminated: False Truncated: False Info: {'prob': 1.0}
State: 8 Action 2 Reward: 0.0 Terminated: False Truncated: False Info: {'prob': 1.0}
State: 9 Action 2 Reward: 0.0 Terminated: False Truncated: False Info: {'prob': 1.0}
State: 10 Action 1 Reward: 0.0 Terminated: False Truncated: False Info: {'prob': 1.0}
State: 14 Action 2 Reward: 1.0 Terminated: True Truncated: False Info: {'prob': 1.0}
0 (0, 1, 0.0)
1 (4, 3, 0.0)
2 (0, 1, 0.0)
3 (4, 1, 0.0)
4 (8, 2, 0.0)
5 (9, 2, 0.0)
6 (10, 1, 0.0)
7 (14, 2, 1.0)
7 (14, 2, 1.0) 7
Returns(14,2) -> [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Update Q(14,2) -> 1.000000
[0.7151168  0.72914016 1.         0.69932166] 2
Update PI(14) -> [0.05 0.05 0.85 0.05]
6 (10, 1, 0.0) 6
Returns(10,1) -> [0.28242953648100017, 0.9, 0.9, 0.9, 0.9, 0.81, 0.9, 0.81, 0.9, 0.9, 0.9, 0.9, 0.7290000000000001, 0.9, 0.9, 0.9, 0.7290000000000001, 0.7290000000000001, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.7290000000000001, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.7290000000000001, 0.9, 0.7290000000000001, 
0.9, 0.9, 0.9, 0.38742048900000015, 0.9, 0.9, 0.9, 0.7290000000000001, 0.9]
Update Q(10,1) -> 0.856015
[0.         0.85601491 0.         0.3298725 ] 1
Update PI(10) -> [0.05 0.85 0.05 0.05]
5 (9, 2, 0.0) 5
Returns(9,2) -> [0.0, 0.25418658283290013, 0.81, 0.81, 0.81, 0.81, 0.7290000000000001, 0.81, 0.7290000000000001, 0.81, 0.81, 0.81, 0.81, 0.6561000000000001, 0.81, 0.0, 0.81, 0.6561000000000001, 0.0, 0.0, 0.81, 0.81, 0.81, 0.81, 0.81, 0.81, 0.81, 0.81, 0.81, 0.81, 0.81, 0.81, 0.0, 0.81, 0.81, 0.6561000000000001, 0.81, 0.81, 0.81, 0.0, 0.5314410000000002, 0.81, 0.6561000000000001, 0.81, 0.81, 0.81, 0.81, 0.81, 0.6561000000000001, 0.81, 0.6561000000000001, 0.81, 0.34867844010000015, 0.81, 0.81, 0.6561000000000001, 0.81]
Update Q(9,2) -> 0.680263
[0.31047964 0.648      0.68026326 0.        ] 2
Update PI(9) -> [0.05 0.05 0.85 0.05]
4 (8, 2, 0.0) 4
Returns(8,2) -> [0.0, 0.0, 0.0, 0.22876792454961012, 0.7290000000000001, 0.7290000000000001, 0.7290000000000001, 0.7290000000000001, 0.5314410000000002, 0.7290000000000001, 0.6561000000000001, 0.7290000000000001, 0.7290000000000001, 0.7290000000000001, 0.7290000000000001, 0.5904900000000002, 0.7290000000000001, 0.0, 0.7290000000000001, 0.47829690000000014, 0.0, 0.0, 0.7290000000000001, 0.7290000000000001, 0.7290000000000001, 0.7290000000000001, 0.7290000000000001, 0.7290000000000001, 0.7290000000000001, 0.7290000000000001, 0.7290000000000001, 0.7290000000000001, 0.7290000000000001, 0.7290000000000001, 0.7290000000000001, 0.7290000000000001, 0.0, 0.7290000000000001, 0.7290000000000001, 0.5904900000000002, 0.7290000000000001, 0.7290000000000001, 0.7290000000000001, 0.0, 0.38742048900000015, 0.7290000000000001, 0.5904900000000002, 0.7290000000000001, 0.7290000000000001, 0.7290000000000001, 0.7290000000000001, 0.7290000000000001, 0.7290000000000001, 0.5904900000000002, 0.7290000000000001, 0.5904900000000002, 0.7290000000000001, 0.31381059609000017, 0.7290000000000001, 0.7290000000000001, 0.7290000000000001, 0.5904900000000002, 0.7290000000000001]
Update Q(8,2) -> 0.595012
[0.50908121 0.         0.59501233 0.19683   ] 2
Update PI(8) -> [0.05 0.05 0.85 0.05]
3 (4, 1, 0.0) 3
Returns(4,1) -> [0.0, 0.0, 0.0, 0.0, 0.0, 0.2058911320946491, 0.0, 0.6561000000000001, 0.0, 0.6561000000000001, 0.6561000000000001, 0.6561000000000001, 0.47829690000000014, 0.6561000000000001, 0.5904900000000002, 0.6561000000000001, 0.5314410000000002, 0.6561000000000001, 0.6561000000000001, 0.5314410000000002, 0.6561000000000001, 0.0, 0.6561000000000001, 0.43046721000000016, 0.0, 0.0, 0.6561000000000001, 0.6561000000000001, 0.6561000000000001, 0.6561000000000001, 0.6561000000000001, 0.6561000000000001, 0.6561000000000001, 0.6561000000000001, 0.6561000000000001, 0.6561000000000001, 0.0, 0.6561000000000001, 0.6561000000000001, 0.5904900000000002, 0.6561000000000001, 0.0, 0.6561000000000001, 0.6561000000000001, 0.5314410000000002, 0.5904900000000002, 0.0, 0.6561000000000001, 0.0, 0.5314410000000002, 0.0, 0.0, 0.31381059609000017, 0.5904900000000002, 0.5314410000000002, 0.6561000000000001, 0.0, 0.6561000000000001, 0.6561000000000001, 0.6561000000000001, 0.6561000000000001, 0.6561000000000001, 0.5314410000000002, 0.6561000000000001, 0.5314410000000002, 0.6561000000000001, 0.28242953648100017, 0.6561000000000001, 0.6561000000000001, 0.5904900000000002, 0.5314410000000002, 0.6561000000000001]
Update Q(4,1) -> 0.460980
[0.20002849 0.46098019 0.         0.21964091] 1
Update PI(4) -> [0.05 0.85 0.05 0.05]
2 (0, 1, 0.0) 0
1 (4, 3, 0.0) 1
Returns(4,3) -> [0.0, 0.0, 0.0, 0.0, 0.16677181699666577, 0.0, 0.38742048900000015, 0.43046721000000016, 0.43046721000000016, 0.0, 0.5314410000000002, 0.47829690000000014, 0.43046721000000016, 0.5314410000000002]
Update Q(4,3) -> 0.241912
[0.20002849 0.46098019 0.         0.24191235] 1
Update PI(4) -> [0.05 0.85 0.05 0.05]
0 (0, 1, 0.0) 0
Returns(0,1) -> [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1500946352969992, 0.0, 0.0, 0.5904900000000002, 0.0, 0.5904900000000002, 0.5904900000000002, 0.5904900000000002, 0.34867844010000015, 0.5904900000000002, 0.5314410000000002, 0.38742048900000015, 0.0, 0.38742048900000015, 0.5904900000000002, 0.5904900000000002, 0.43046721000000016, 0.5904900000000002, 0.0, 
0.5904900000000002, 0.0, 0.38742048900000015, 0.0, 0.0, 0.0, 0.47829690000000014, 0.43046721000000016, 0.5904900000000002, 0.5904900000000002, 0.5904900000000002, 0.5904900000000002, 0.5904900000000002, 0.5904900000000002, 0.5904900000000002, 0.5904900000000002, 0.0, 0.0, 0.5904900000000002, 0.5904900000000002, 0.5314410000000002, 0.5904900000000002, 0.0, 0.5904900000000002, 0.5314410000000002, 0.47829690000000014, 0.5314410000000002, 0.0, 0.5904900000000002, 0.0, 0.47829690000000014, 0.0, 0.0, 0.0, 0.28242953648100017, 0.5314410000000002, 0.47829690000000014, 0.5904900000000002, 0.0, 0.5904900000000002, 0.5904900000000002, 0.5904900000000002, 0.5904900000000002, 0.5904900000000002, 0.47829690000000014, 0.0, 0.5904900000000002, 0.0, 0.47829690000000014, 0.5904900000000002, 0.25418658283290013, 0.5904900000000002, 0.5904900000000002, 0.47829690000000014, 0.38742048900000015, 0.47829690000000014]
Update Q(0,1) -> 0.347292
[0.19367946 0.34729236 0.17780244 0.09356642] 1
Update PI(0) -> [0.05 0.85 0.05 0.05]
'''

Episode 0/500 --------------------- 
--> Reward: 0.0, #Steps: 100,                 Avg Rewards: 0.0, Avg steps: 100.0
Episode 1/500 --------------------- 


KeyboardInterrupt: 

# readchar이용 사람조정 버전
- command에서 실행해야 조정가능

In [1]:
import gym
import readchar

#MACROS
LEFT = 0
DOWN = 1
RIGHT = 2
UP = 3
 
# Key mapping
arrow_keys = {
    'w' : UP,
    's' : DOWN,
    'd' : RIGHT,
    'a' : LEFT
}
 
env = gym.make('FrozenLake-v1', desc=None, map_name="4x4", is_slippery=False, render_mode='human')
state = env.reset()

while True:    
    # Choose an action from keyboard
    key = readchar.readkey()  
    if key not in arrow_keys.keys():
        
        print("Game aborted!")
        break
 
    action = arrow_keys[key]
    # print(key, arrow_keys[key])
    state, reward, done, truncated, info= env.step(action)       

    env.render() # Show the board after action
    print("State:", state, "Action", action, "Reward:", reward, "Info:", info)
 
    if done: 
        print("Finished with reward", reward)
        break